In [2]:
import pandas as pd
import matplotlib as plt

#Define filepaths
path_coe = 'data/COEBiddingResultsPrices.csv'
path_deregistered = 'data/MotorVehiclesDeRegisteredUnderVehicleQuotaSystemMonthly.csv'
path_registered = 'data/NewRegistrationOfMotorVehiclesUnderVehicleQuotaSystemMonthly.csv'
path_cpi = 'data/outputFile.xlsx'
path_nom_household = 'data/NominalChangeinAverageMonthlyHouseholdEmploymentIncomePerHouseholdMemberExcludingEmployerCPFContributionsAmongResidentEmployedHouseholdsbyDecilesHouseholdEmploymentIncomeAnnual20002024.csv'
#path_real_household = 'data/RealChangeinMonthlyHouseholdEmploymentIncomePerHouseholdMemberExcludingEmployerCPFContributionsAmongResidentEmployedHouseholdsatSelectedPercentilesHouseholdEmploymentIncomeAnnual20002024.csv'

#Load datasets
coe = pd.read_csv(path_coe, thousands=',') #thousands separator causing issues
deregistered = pd.read_csv(path_deregistered)
registered = pd.read_csv(path_registered)
cpi = pd.read_excel(path_cpi, sheet_name='T7', skiprows=10) #the only xlsx file
nom_household = pd.read_csv(path_nom_household)

#Function to convert date format to YYYY-MM in digits
def convert_date_format(x, original_months, converted_months):
    ref_dict = dict(zip(original_months, converted_months))
    this_date = x[5:] #start the search from the 5th character to skip the year and space
    return f'{x[:4]}-{ref_dict[this_date]}'

#Function to rebase values to their corresponding month in a base year
def rebase_index(df, year_base, year_start, year_end_excl, label_time, label_value, list_months): #only for unique year-months
    list_years = [i for i in range(year_start,year_end_excl)]
    for month in list_months:
        base_year_month_value = float(df.loc[df[label_time] == f'{year_base}-{month}',label_value].iloc[0])
        for year in list_years:
            try:
                current_value = float(df.loc[df[label_time] == f'{year}-{month}',label_value].iloc[0])
                rebased_value = round((current_value / base_year_month_value) * 100,3)
                df.loc[df[label_time] == f'{year}-{month}', f'{label_value}_rebased'] = rebased_value
            except: #dates may not be found
                print(f'Error for {year} {month}: {df.loc[df[str(label_time)] == f'{year}-{month}']}')
                pass
    return df



In [14]:
filtereddf_rebased

,month,vehicle_class,quota,bids_success,bids_received,premium,premium_rebased
monthandcategory,,,,,,,
2010-01Category D,2010-01,Category D,375.5,371.5,530.0,870.5,100.000
2010-02Category D,2010-02,Category D,367.0,352.0,498.5,926.5,100.000
2010-03Category D,2010-03,Category D,378.0,375.5,481.0,1179.5,100.000
2010-04Category D,2010-04,Category D,338.5,322.5,396.0,1237.0,100.000
2010-05Category D,2010-05,Category D,352.0,338.5,441.5,1282.0,100.000
...,...,...,...,...,...,...,...
2025-07Category D,2025-07,Category D,536.5,533.5,625.5,9450.0,724.138
2025-08Category D,2025-08,Category D,536.5,530.5,643.5,8999.0,705.251
2025-09Category D,2025-09,Category D,539.5,539.5,625.0,9155.0,619.838


In [17]:
##### NOMINAL COE DATA #####
#cut off at last date of Nov 2025
index_of_2025Nov = coe[coe['month'] == '2025-11'].index[-1] #get the last occurrence
coe_nov25 = coe.loc[:index_of_2025Nov]
#filter down to cars A, cars B, and bikes
coe_nov25_filtered = coe_nov25[coe_nov25['vehicle_class'].isin(['Category A','Category B', 'Category D'])]
#coe_nov25_filtered[['quota', 'bids_success', 'bids_received', 'premium']] = coe_nov25_filtered[['quota', 'bids_success', 'bids_received', 'premium']].astype('int64') #in case need to convert to int
coe_nov25_filtered_bidavg = coe_nov25_filtered.groupby(['month','vehicle_class']).mean().reset_index().drop(columns=['bidding_no']) #average bids 1 and 2
#work on index
coe_nov25_filtered_bidavg.columns = coe_nov25_filtered_bidavg.columns.to_flat_index() #flatten multiindex columns
coe_nov25_filtered_bidavg['monthandcategory'] = coe_nov25_filtered_bidavg['month'] + coe_nov25_filtered_bidavg['vehicle_class'] #new index column
coe_nov25_filtered_bidavg = coe_nov25_filtered_bidavg.set_index(['monthandcategory'])
#re-base to 2010
iterate_through_vehicle_class = ['Category A', 'Category B', 'Category D']
for category in iterate_through_vehicle_class:
    filtereddf_rebased = rebase_index(coe_nov25_filtered_bidavg.loc[coe_nov25_filtered_bidavg['vehicle_class'] == category], 2010, 2010, 2026, 'month', 'premium', numerical_months)
    #coe_nov25_filtered_bidavg.update(filtereddf_rebased)
    coe_nov25_filtered_bidavg = pd.merge(coe_nov25_filtered_bidavg, filtereddf_rebased, on=['monthandcategory'], how='left')

#coe_nov25_filtered_bidavg_rebased = rebase_index(coe_nov25_filtered_bidavg, 2010, 2010, 2026, 'month', 'premium', numerical_months)
coe_nov25_filtered_bidavg

C:\Users\user\AppData\Local\Temp\ipykernel_7256\2708304530.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df[label_time] == f'{year}-{month}', f'{label_value}_rebased'] = rebased_value


Error for 2020 04: Empty DataFrame
Columns: [month, vehicle_class, quota, bids_success, bids_received, premium, premium_rebased]
Index: []
Error for 2020 05: Empty DataFrame
Columns: [month, vehicle_class, quota, bids_success, bids_received, premium, premium_rebased]
Index: []
Error for 2020 06: Empty DataFrame
Columns: [month, vehicle_class, quota, bids_success, bids_received, premium, premium_rebased]
Index: []
Error for 2025 12: Empty DataFrame
Columns: [month, vehicle_class, quota, bids_success, bids_received, premium, premium_rebased]
Index: []


KeyError: 'vehicle_class'

In [3]:
def plot_twin_axis(x, y1, y2, y1_label, y2_label, title):
    fig, ax1 = plt.subplots()

    colour_ax1 = 'tab:blue'
    ax1.set_xlabel('Month')
    ax1.set_ylabel(y1_label, color=colour_ax1)
    ax1.plot(x, y1, color=colour_ax1)
    ax1.tick_params(axis='y', labelcolor=colour_ax1)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    colour_ax2 = 'tab:red'
    ax2.set_ylabel(y2_label, color=colour_ax2)  # we already handled the x-label with ax1
    ax2.plot(x, y2, color=colour_ax2)
    ax2.tick_params(axis='y', labelcolor=colour_ax2)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.title(title)
    plt.show()

In [ ]:
df_plot = coe_nov25_filtered_bidavg
plot_twin_axis(df_plot[df_plot['vehicle_class']=='Category A']['month'], df_plot[df_plot['vehicle_class']=='Category A']['month'], df_plot[df_plot['vehicle_class']=='Category B']['month'], y1_label, y2_label, title)

In [ ]:
coe_nov25_filtered_bidavg[['month','vehicle_class','premium']]#.plot()

,month,vehicle_class,premium
0,2010-01,Category A,19501.5
1,2010-01,Category B,20795.0
2,2010-01,Category D,870.5
3,2010-02,Category A,20164.5
4,2010-02,Category B,23534.5
...,...,...,...
559,2025-10,Category B,136444.5
560,2025-10,Category D,9599.5
561,2025-11,Category A,109501.0
562,2025-11,Category B,122445.5


In [4]:
cpi_t = cpi.transpose()
cpi_t.columns = cpi_t.iloc[0]
cpi_t = cpi_t[1:]
cpi_t.reset_index(inplace=True)
cpi_t.rename(columns={'index': 'Period'}, inplace=True)
cpi_mainmetric = cpi_t[['Period','All Items']] #only interested in 'All Items'
cpi_mainmetric_sorted = cpi_mainmetric[::-1] #reverse the order to have earliest month first
#cut off at earliest date of coe dataset
index_of_earliestdate = cpi_mainmetric_sorted[cpi_mainmetric_sorted['Period'] == '2010 Jan'].index[0]
cpi_mainmetric_sorted_earliestdate = cpi_mainmetric_sorted.loc[index_of_earliestdate:]
#convert date format to YYYY-MM
text_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
numerical_months = ['01','02','03','04','05','06','07','08','09','10','11','12']
cpi_mainmetric_sorted_earliestdate.loc[:,'Period'] = cpi_mainmetric_sorted_earliestdate.loc[:,'Period'].apply(convert_date_format, args=(text_months, numerical_months))
#re-base to 2010 jan = 100
cpi_mainmetric_sorted_earliestdate_rebased = rebase_index(cpi_mainmetric_sorted_earliestdate, 2010, 2010, 2026, 'Period', 'All Items', numerical_months)
cpi_mainmetric_sorted_earliestdate_rebased

C:\Users\user\AppData\Local\Temp\ipykernel_7256\2708304530.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df[label_time] == f'{year}-{month}', f'{label_value}_rebased'] = rebased_value


Error for 2025 12: Empty DataFrame
Columns: [Period, All Items, All Items_rebased]
Index: []


Data Series,Period,All Items,All Items_rebased
190,2010-01,73.64,100.000
189,2010-02,73.771,100.000
188,2010-03,74,100.000
187,2010-04,74.596,100.000
186,2010-05,75.025,100.000
...,...,...,...
4,2025-07,100.704,133.806
3,2025-08,100.742,133.243
2,2025-09,101.099,133.591
1,2025-10,101.589,133.686
